# <center>CryptoCurrencies CA#2<center>
<h1><center>Transacting on Bitcoin TestNet / Firing up an Ethereum Node<center></h1>

#### <center>"Mahdi - Noori"<center>
#### <center>"810100231"<center>

## Imports

In [1]:
import hashlib
import base58
import codecs
import os
import ecdsa
import secrets
import bitcoin

### 1. Address Generation:

 1.1: Base58 and WIF address

In [2]:
def private_key_to_wif(private_key):

    # Add testnet prefix to the private key
    private_key = b'\xef' + private_key
    
    sha256 = hashlib.sha256(private_key).digest()    
    sha256 = hashlib.sha256(sha256).digest()    
    checksum = sha256[:4]    
    private_key = private_key + checksum
    
    wif_private_key = base58.b58encode(private_key)
    return wif_private_key.decode('utf-8')

def generate_testnet_address():
    private_key = secrets.token_bytes(32)    
    wif_private_key = private_key_to_wif(private_key)
    
    public_key = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1).verifying_key
    if public_key is None:
        raise ValueError("Invalid public key")
    
    public_key = (b"\x04" + public_key.to_string()) 
    
    sha256 = hashlib.sha256(public_key).digest()
    ripemd160 = hashlib.new("ripemd160")
    ripemd160.update(sha256)
    
    # Add testnet prefix to the public key hash
    public_key = b'\x6f' + ripemd160.digest()
    
    sha256_1 = hashlib.sha256(public_key).digest()
    sha256_2 = hashlib.sha256(sha256_1).digest()    
    checksum = sha256_2[:4]
    
    public_key = public_key + checksum
    address = base58.b58encode(public_key)
    
    return address.decode('utf-8'), wif_private_key

In [3]:
address, wif_private_key = generate_testnet_address()

print("WIF Private Key:", wif_private_key)
print("Address:", address)

WIF Private Key: 91mM5mjYb5QpAtSVBx1n2Uaj9XyM4Wfj87g9Vqwd1YnEkswHw4w
Address: mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ


The main difference between testnet and mainnet addresses lies in the networks they operate on and their purpose:

1. **Mainnet Addresses**: Mainnet addresses are used in the Bitcoin main network. They represent real, live Bitcoin transactions and carry actual economic value. Mainnet addresses start with the number "1" for Pay to Public Key Hash (P2PKH) addresses and with the number "3" for Pay to Script Hash (P2SH) addresses.

2. **Testnet Addresses**: Testnet addresses, on the other hand, are used in the Bitcoin test network (testnet). The testnet is a separate network that mimics the main network but is designed for testing and development purposes. Testnet addresses and testnet bitcoins have no value in the real-world economy and are used for experimentation, testing, and learning. Testnet addresses start with the letter "m" or "n" for P2PKH addresses and with the letters "2" or "3" for P2SH addresses.

It's important to note that testnet and mainnet addresses are not compatible with each other. Testnet bitcoins cannot be used or transferred to mainnet addresses, and vice versa. They exist on separate networks to maintain the distinction between real economic transactions and testing environments.

1.2: Vanity Address Generator


In [4]:
def generate_vanity_address(prefix):
    vanity_address = ""
    number_of_tries = 0

    while True:
        address, _ = generate_testnet_address()
        number_of_tries += 1
        if address[1:].startswith(prefix):
            vanity_address = address
            break
    return vanity_address, number_of_tries

vanity_address, number_of_tries = generate_vanity_address("4ab")
print(f"Vanity Address {vanity_address} created after {number_of_tries} tries")    

Vanity Address n4ab8sWCxNgczrs3xZHwfNo5MjEbHfRRcA created after 91902 tries


### 2. Transacting on Bitcoin TestNet:

#### Utility Functions:

In [2]:
import requests

from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH

def send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey):
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    txin = create_txin(txid_to_spend, utxo_index)
    new_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)
    return broadcast_transaction(new_tx)


def create_txin(txid, utxo_index):
    return CMutableTxIn(COutPoint(lx(txid), utxo_index))


def create_txout(amount, scriptPubKey):
    return CMutableTxOut(amount*COIN, CScript(scriptPubKey))


def create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, seckey):
    tx = CMutableTransaction([txin], txout)
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)
    sig = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return sig


def create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig):
    tx = CMutableTransaction([txin], txout)
    txin.scriptSig = CScript(txin_scriptSig)
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))
    return tx


def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://api.blockcypher.com/v1/btc/test3/txs/push',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction,
    )


In [3]:
import bitcoin.wallet
from bitcoin.core import COIN, b2lx, serialize, x, lx, b2x
bitcoin.SelectParams("testnet")

def P2PKH_scriptPubKey(address):
    return [OP_DUP, OP_HASH160, Hash160(address), OP_EQUALVERIFY, OP_CHECKSIG]


def P2PKH_scriptSig(txin, txout, txin_scriptPubKey, private_key, public_key):

    signature = create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, private_key)
    return [signature, public_key]

def print_response(response, address, public_key, private_key):
    print(address)  # Prints your address in base58
    print(public_key.hex())  # Print your public key in hex
    print(private_key.hex())  # Print your private key in hex
    print(response.status_code, response.reason)
    print(response.text)  # Report the hash of transaction which is printed in this section result

Now use the provided functions to make these transactions:

Note that the first stream of bitcoins in the first place that we got from the faucet has txid of $$7990000f3f7276a3260d045b97d8c680f15e17d0e6f4f95e904986f82a8c55a6$$
We recieved 0.006 coins here.

2.1: One input and two outputs transaction

Here is the address that we use
The private key is:
$$ 91mM5mjYb5QpAtSVBx1n2Uaj9XyM4Wfj87g9Vqwd1YnEkswHw4w $$

In [4]:
my_private_key = bitcoin.wallet.CBitcoinSecret("91mM5mjYb5QpAtSVBx1n2Uaj9XyM4Wfj87g9Vqwd1YnEkswHw4w")
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)
print("My Address:", my_address)

My Address: mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ


For the first transaction we set the fee 0.00045 and 0.0055 for free for all usage and 0.00005 for burning.

In [6]:
# First Transaction

def send_from_P2PKH_transaction_to_two_outputs(first_amount_to_send, second_amount_to_send, txid_to_spend, utxo_index):
    first_txout = create_txout(first_amount_to_send, [OP_FALSE])
    second_txout = create_txout(second_amount_to_send, [OP_TRUE])

    txout = [first_txout, second_txout]
    txin_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig(txin, txout, txin_scriptPubKey, my_private_key, my_public_key)

    sig_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(sig_tx)

first_amount_to_send = 0.00005
second_amount_to_send = 0.0055
txid_to_spend = ('7990000f3f7276a3260d045b97d8c680f15e17d0e6f4f95e904986f82a8c55a6')
utxo_index = 0
print_response(send_from_P2PKH_transaction_to_two_outputs(first_amount_to_send, second_amount_to_send, txid_to_spend, utxo_index), my_address, my_public_key, my_private_key)

mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ
04c6f372f079ebc9fb5fede0fa2d929471cfbc612feb92cf0d269ee5e47826c8b35aa758b6f0defbafc608acaa4f510279d963467fbd82491902fef478d6d8d4a5
17ae8aa6337fa2e0e463904d77a44fcd9dd3005e6fa989875382604e74733fb5
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "e9022c5923ad92a944ecb1f26048b549ba7197eabc699ce56e73e7b3c6ceb680",
    "addresses": [
      "mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ"
    ],
    "total": 555000,
    "fees": 65000,
    "size": 209,
    "vsize": 209,
    "preference": "high",
    "relayed_by": "104.244.77.108",
    "received": "2024-05-25T19:03:02.041046113Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 2,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "7990000f3f7276a3260d045b97d8c680f15e17d0e6f4f95e904986f82a8c55a6",
        "output_index": 0,
        "script": "473044022059e1596c967f2a9c1bad486ad9d7bc23a37c6b9b89e1f6950ba16668d191fb280220202922b9567ec3f31be704057

For the second one we can send 0.0055 back and by setting the transaction fee to 0.00045 we can send back 0.00505 coins.

In [8]:
# Second Transaction

def send_P2PKH_tx_that_anyone_can_spend(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey):
    txout = create_txout(amount_to_spend, txout_scriptPubKey)
    txin = create_txin(txid_to_spend, utxo_index)

    txin_scriptPubKey = [OP_TRUE]
    txin_scriptSig = []
    sig_tx = create_signed_transaction(txin, [txout], txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(sig_tx)

amount_to_send = 0.00505
txid_to_spend = ('e9022c5923ad92a944ecb1f26048b549ba7197eabc699ce56e73e7b3c6ceb680')
utxo_index = 1
txout_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
print_response(send_P2PKH_tx_that_anyone_can_spend(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey), my_address, my_public_key, my_private_key)

mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ
04c6f372f079ebc9fb5fede0fa2d929471cfbc612feb92cf0d269ee5e47826c8b35aa758b6f0defbafc608acaa4f510279d963467fbd82491902fef478d6d8d4a5
17ae8aa6337fa2e0e463904d77a44fcd9dd3005e6fa989875382604e74733fb5
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "8e27cc9c57399ae487d1e3c2140fd642cbb38affa4bc9adea818c140ffdb7a5c",
    "addresses": [
      "mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ"
    ],
    "total": 505000,
    "fees": 45000,
    "size": 85,
    "vsize": 85,
    "preference": "high",
    "relayed_by": "104.244.77.108",
    "received": "2024-05-26T07:43:41.092323413Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "e9022c5923ad92a944ecb1f26048b549ba7197eabc699ce56e73e7b3c6ceb680",
        "output_index": 1,
        "output_value": 550000,
        "sequence": 4294967295,
        "script_type": "unknown",
        "age": 2817766
   

As you can see in first transaction by setting one of outputs to ```OP_TRUE``` anyone can spend the amount and setting the other one to ```OP_FALSE``` no one can spend it.
<br><br> In the second one we send the spendable amount in first part to our account. Notice that for using it we are not use any signatures because it is open to all.   

2.2: Pay to MultiSig transaction (P2MS)


First we generate the addresses

In [6]:
private_keys = []

for i in range(3):
    address, wif_private_key = generate_testnet_address()

    print("WIF Private Key:", wif_private_key)
    private_keys.append(wif_private_key)
    print("Address:", address)

WIF Private Key: 92D2hRT2V851obvrW7TKP2ffGn7DVd8EE682n6YgzG6tubh36sD
Address: msTsTugBoWn8HftMNEGPzzhrq1XfPur5B1
WIF Private Key: 92gSw5M4ZwR8yeRLMuEFRBmLgTjxV7RtySSnDHRbEU3R8nC6kkG
Address: mw9AE5jCA2q5LMjvoPKyFc569CPsh2qeoe
WIF Private Key: 93Ks9oD5QTkBKJnxy5aYE4Fu3uABzksJsLwpRDx9weXaLerz7T3
Address: n1zHy3YwR3FqKXuK9dLy4jBVBBfcjj4jzq


In [7]:
private_key1= bitcoin.wallet.CBitcoinSecret('92D2hRT2V851obvrW7TKP2ffGn7DVd8EE682n6YgzG6tubh36sD')
public_key1 = private_key1.pub

private_key2 = bitcoin.wallet.CBitcoinSecret('92gSw5M4ZwR8yeRLMuEFRBmLgTjxV7RtySSnDHRbEU3R8nC6kkG')
public_key2 = private_key2.pub

private_key3 = bitcoin.wallet.CBitcoinSecret('93Ks9oD5QTkBKJnxy5aYE4Fu3uABzksJsLwpRDx9weXaLerz7T3')
public_key3 = private_key3.pub

From the previous part we have 0.00505 coins. given the 0.00045 fee we can send 0.0046 coins here.

In [9]:
# First Transaction

def send_P2PKH_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey):
    txin = create_txin(txid_to_spend, utxo_index)
    txout = create_txout(amount_to_spend, txout_scriptPubKey)

    txin_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txin_scriptSig = P2PKH_scriptSig(txin, [txout], txin_scriptPubKey, my_private_key, my_public_key)
    sig_tx = create_signed_transaction(txin, [txout], txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(sig_tx)

amount_to_spend = 0.0046
txid_to_spend = ('8e27cc9c57399ae487d1e3c2140fd642cbb38affa4bc9adea818c140ffdb7a5c')
utxo_index = 0
txout_scriptPubKey = [OP_2, public_key1, public_key2, public_key3, OP_3, OP_CHECKMULTISIG]
print_response(send_P2PKH_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey), my_address, my_public_key, my_private_key)

mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ
04c6f372f079ebc9fb5fede0fa2d929471cfbc612feb92cf0d269ee5e47826c8b35aa758b6f0defbafc608acaa4f510279d963467fbd82491902fef478d6d8d4a5
17ae8aa6337fa2e0e463904d77a44fcd9dd3005e6fa989875382604e74733fb5
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "0a57b593553ec4bda3057289027ee5a4e8e282c5128751d8b4872c24acf98abf",
    "addresses": [
      "mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ",
      "zNNG7fpaMaAMAbuCkJFkHv4GEGso36hJRp"
    ],
    "total": 460000,
    "fees": 45000,
    "size": 400,
    "vsize": 400,
    "preference": "high",
    "relayed_by": "104.244.77.108",
    "received": "2024-05-26T14:09:27.966519587Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "8e27cc9c57399ae487d1e3c2140fd642cbb38affa4bc9adea818c140ffdb7a5c",
        "output_index": 0,
        "script": "4830450221008d4c26e3abb4eebc17b80eb9309295d3040391d9515fe73

From the previous part we have 0.0046 coins and given the 0.00045 fee here we can transfer 0.00415 coins.

In [9]:
# Second Transaction
def P2PKH_scriptSig_with_two_signers(txin, txout, txin_scriptPubKey, first_private_key, second_private_key):
    first_signature = create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, first_private_key)
    second_signature = create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, second_private_key)
    return [OP_0, first_signature, second_signature]

def send_P2PKH_multi_sig_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey):
    txin = create_txin(txid_to_spend, utxo_index)
    txout = create_txout(amount_to_spend, txout_scriptPubKey)

    txin_scriptPubKey = [OP_2, public_key1, public_key2, public_key3, OP_3, OP_CHECKMULTISIG]
    txin_scriptSig = P2PKH_scriptSig_with_two_signers(txin, [txout], txin_scriptPubKey, private_key1, private_key2)
    sig_tx = create_signed_transaction(txin, [txout], txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(sig_tx)

amount_to_spend = 0.00415
txid_to_spend = ('0a57b593553ec4bda3057289027ee5a4e8e282c5128751d8b4872c24acf98abf')
utxo_index = 0
txout_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
print_response(send_P2PKH_multi_sig_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey), my_address, my_public_key, my_private_key)

mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ
04c6f372f079ebc9fb5fede0fa2d929471cfbc612feb92cf0d269ee5e47826c8b35aa758b6f0defbafc608acaa4f510279d963467fbd82491902fef478d6d8d4a5
17ae8aa6337fa2e0e463904d77a44fcd9dd3005e6fa989875382604e74733fb5
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "1de7e090773e8b99e7647d0d9bdc05846352a124298cc2992df37505599c9338",
    "addresses": [
      "zNNG7fpaMaAMAbuCkJFkHv4GEGso36hJRp",
      "mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ"
    ],
    "total": 415000,
    "fees": 45000,
    "size": 231,
    "vsize": 231,
    "preference": "high",
    "relayed_by": "188.118.98.114",
    "received": "2024-05-26T17:57:24.284823372Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "0a57b593553ec4bda3057289027ee5a4e8e282c5128751d8b4872c24acf98abf",
        "output_index": 0,
        "script": "00483045022100c86320f2154f2de400e8915b2de374b2cab998f91c408

Here when we are sending the money, we put the limitation 
``` [OP_2, public_key1, public_key2, public_key3, OP_3, OP_CHECKMULTISIG] ```
causing that the spender should have two of these three signatures so it can spend it.
<br>
We create these two signatures using function `P2PKH_scriptSig_with_two_signers` which gets two private keys and signs the transaction with each one.

2.3: Custom Transaction (BitCoin Scripting)


First we define our constants for the problem

In [5]:
BIRTH_YEAR = 2003
CURR_YEAR = 2024
PASSWORD = 810100231

From previous section we have 0.00415 coins and given the fact that we give 0.00045 fee we can send 0.0037 here.

In [17]:
# First Transaction

def send_P2PKH_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey):
    txin = create_txin(txid_to_spend, utxo_index)
    txout = create_txout(amount_to_spend, txout_scriptPubKey)

    txin_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txin_scriptSig = P2PKH_scriptSig(txin, [txout], txin_scriptPubKey, my_private_key, my_public_key)
    tx = create_signed_transaction(txin, [txout], txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(tx)

amount_to_spend = 0.0037
txid_to_spend = ('1de7e090773e8b99e7647d0d9bdc05846352a124298cc2992df37505599c9338')
utxo_index = 0
txout_scriptPubKey = [OP_SUB, 18 , OP_GREATERTHAN, OP_IF, OP_DROP, OP_TRUE, OP_ELSE, OP_HASH160, 
                      Hash160(PASSWORD.to_bytes(4, byteorder="little")), OP_EQUAL, OP_ENDIF]
print_response(send_P2PKH_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey), my_address, my_public_key, my_private_key)

mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ
04c6f372f079ebc9fb5fede0fa2d929471cfbc612feb92cf0d269ee5e47826c8b35aa758b6f0defbafc608acaa4f510279d963467fbd82491902fef478d6d8d4a5
17ae8aa6337fa2e0e463904d77a44fcd9dd3005e6fa989875382604e74733fb5
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "a1cafeba32d4901cd64ba71289231818288ceb4b451f196a65a62555bd074a7d",
    "addresses": [
      "mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ"
    ],
    "total": 370000,
    "fees": 45000,
    "size": 230,
    "vsize": 230,
    "preference": "high",
    "relayed_by": "104.244.77.108",
    "received": "2024-05-27T15:13:15.502246215Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "1de7e090773e8b99e7647d0d9bdc05846352a124298cc2992df37505599c9338",
        "output_index": 0,
        "script": "4730440220326795257a2c6fc47ce860d699b5ac4a7e6d08e357e670ee6f7e31a7a28ff51602201866f8872cf463e218f36d414

For the stack logic here we setup the stack like `[password, current-year, birth-year]`. Now first we check that if the age is above 18. By first sub the top two elements of stack using `OP_SUB` and then putting 18 on stack. Then using `OP_GREATERTHAN` we check the age. If it was true we drop the password using `OP_DROP` and then put true on top.

If it wasn't greater we first hash the password using `OP_HASH` and then check it with the real password that we have which is `PASSWORD`.

Now we define new values for test.
Here we want to see if the age condition passes.
From the prev section we have 0.0037 and the fee is 0.00045 so we can transfer 0.00325.

Below are the new values for the validation

In [6]:
NEW_BIRTH_YEAR = 2003
NEW_CURR_YEAR = 2024
NEW_PASSWORD = 810100232

Here we send the coins to a key that has the restrictions of the description again.

In [8]:
# Second Transaction

def send_P2PKH_custome_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey):
    txin = create_txin(txid_to_spend, utxo_index)
    txout = create_txout(amount_to_spend, txout_scriptPubKey)

    txin_scriptPubKey = [OP_SUB, 18 , OP_GREATERTHAN, OP_IF, OP_DROP, OP_TRUE, OP_ELSE, OP_HASH160, 
                      Hash160(PASSWORD.to_bytes(4, byteorder="little")), OP_EQUAL, OP_ENDIF]
    txin_scriptSig = [NEW_PASSWORD, NEW_CURR_YEAR, NEW_BIRTH_YEAR]
    tx = create_signed_transaction(txin, [txout], txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(tx)

amount_to_spend = 0.00325
txid_to_spend = ('a1cafeba32d4901cd64ba71289231818288ceb4b451f196a65a62555bd074a7d')
utxo_index = 0
txout_scriptPubKey = [OP_SUB, 18 , OP_GREATERTHAN, OP_IF, OP_DROP, OP_TRUE, OP_ELSE, OP_HASH160, 
                      Hash160(PASSWORD.to_bytes(4, byteorder="little")), OP_EQUAL, OP_ENDIF]
print_response(send_P2PKH_custome_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey), my_address, my_public_key, my_private_key)

mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ
04c6f372f079ebc9fb5fede0fa2d929471cfbc612feb92cf0d269ee5e47826c8b35aa758b6f0defbafc608acaa4f510279d963467fbd82491902fef478d6d8d4a5
17ae8aa6337fa2e0e463904d77a44fcd9dd3005e6fa989875382604e74733fb5
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "ef51b3d5528c2631b075a557688baeb62ac7674fef407936f7615cb7529d2dc0",
    "addresses": [],
    "total": 325000,
    "fees": 45000,
    "size": 103,
    "vsize": 103,
    "preference": "high",
    "relayed_by": "104.244.77.108",
    "received": "2024-05-27T19:08:59.818390777Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "a1cafeba32d4901cd64ba71289231818288ceb4b451f196a65a62555bd074a7d",
        "output_index": 0,
        "script": "040826493002e80702d307",
        "output_value": 370000,
        "sequence": 4294967295,
        "script_type": "unknown",
        "age": 2818101
     

Now we define new values for second test.
Here we want to see if the password condition passes.
From the prev section we have 0.00325 and the fee is 0.00045 so we can transfer 0.0028.

In [6]:
NEW_BIRTH_YEAR = 2010
NEW_CURR_YEAR = 2024
NEW_PASSWORD = 810100231

In [8]:
# Third Transaction

def send_P2PKH_custome_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey):
    txin = create_txin(txid_to_spend, utxo_index)
    txout = create_txout(amount_to_spend, txout_scriptPubKey)

    txin_scriptPubKey = [OP_SUB, 18 , OP_GREATERTHAN, OP_IF, OP_DROP, OP_TRUE, OP_ELSE, OP_HASH160, 
                      Hash160(PASSWORD.to_bytes(4, byteorder="little")), OP_EQUAL, OP_ENDIF]
    txin_scriptSig = [NEW_PASSWORD, NEW_CURR_YEAR, NEW_BIRTH_YEAR]
    tx = create_signed_transaction(txin, [txout], txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(tx)


amount_to_spend = 0.0028
txid_to_spend = ('ef51b3d5528c2631b075a557688baeb62ac7674fef407936f7615cb7529d2dc0')
utxo_index = 0
txout_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
print_response(send_P2PKH_custome_tx(amount_to_spend, txid_to_spend, utxo_index, txout_scriptPubKey), my_address, my_public_key, my_private_key)

mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ
04c6f372f079ebc9fb5fede0fa2d929471cfbc612feb92cf0d269ee5e47826c8b35aa758b6f0defbafc608acaa4f510279d963467fbd82491902fef478d6d8d4a5
17ae8aa6337fa2e0e463904d77a44fcd9dd3005e6fa989875382604e74733fb5
201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "f3106a88d621b17b45089a182953341265700346a2fe96c16abc0c91455d2e61",
    "addresses": [
      "mx6bnbmqcpcL8xTtqCnrwARmyBexsSAquJ"
    ],
    "total": 280000,
    "fees": 45000,
    "size": 96,
    "vsize": 96,
    "preference": "high",
    "relayed_by": "104.244.77.108",
    "received": "2024-05-27T20:25:59.774046734Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "ef51b3d5528c2631b075a557688baeb62ac7674fef407936f7615cb7529d2dc0",
        "output_index": 0,
        "script": "040726493002e80702da07",
        "output_value": 325000,
        "sequence": 4294967295,
        "script_

## Q1:

Proof of burn (POB) is an alternative consensus algorithm that tries to address the high energy consumption issue of a POW system.

POB is often called a POW system without energy waste. It operates on the principle of allowing miners to burn virtual currency tokens. They are then granted the right to write blocks in proportion to the coins burnt.

To burn the coins, miners send them to a verifiably un-spendable address. This process does not consume many resources and ensures that the network remains active and agile. Depending upon the implementation, miners are allowed to burn the native currency or the currency of an alternate chain, such as Bitcoin. In exchange, they receive a reward in the native currency token of the blockchain.

To prevent the possibility of unfair advantages for early adopters the POB system has implemented a mechanism that promotes the periodic burning of cryptocurrency coins to maintain mining power. The power of burnt coins reduces partially each time a new block is mined. This promotes regular activity by the miner.

The other thing to notice here is that burning coins can cause more justice in the network because the power in consensus does not just depend on mining power of miners.

Burning coins reduces the total supply of bitcoins. This reduction acts like remurrage on the remaining coins increasing their value proportionally. For example, if 21 million bitcoins exist and 1 million are burned, the remaining 20 million bitcoins effectively increase in value as if multiplied by 21/20.

In a world without explicit remurrage, the real value of holdings increases quietly due to the reduced supply. Holders can spend a portion of their holdings that represents this value increase without losing their original share of the total supply.

## Q2:
Bitcoin's scripting language is a stack-based language designed to process transactions within the Bitcoin network. It plays a crucial role in the validation of transactions and the execution of various types of transactions including multi-signature transactions and time-locked transactions.

Here are some of the features:<br>

1. `Stack-Based Execution:` Script operates using a stack-based architecture where operations push and pop data from a stack.

2. `Simple and Limited:` Script is intentionally not Turing complete. This means it lacks certain features such as loops and more complex control flow mechanisms. The primary reason for this is to maintain security and prevent infinite loops or other problematic behavior that could arise from more complex scripts.

3. `Deterministic Execution:` Every node in the Bitcoin network must reach the same conclusion about whether a transaction is valid. So Script is designed to be deterministic which means that the same script will always produce the same result given the same input.

**Transactions in Bitcoin involve two types of scripts:**<br>

- `Locking Script (ScriptPubKey):` 

This script specifies the conditions that must be met for someone to spend the bitcoins.

- `Unlocking Script (ScriptSig):` 

This script provides the data required to satisfy the conditions set by the locking script.

This language is not turing complete because Script does not support loops which prevents the possibility of infinite loops. The other thing to notice here is that by limiting the complexity of scripts it ensures that transaction validation remains efficient.

**Potential benefits if it was turing complete:**

A Turing complete language could enable a wider variety of complex applications. This could include more sophisticated multi-signature schemes and etc.By allowing more complex scripts Bitcoin could compete more directly with other blockchain platforms like Ethereum which are designed with Turing complete languages.